<a href="https://colab.research.google.com/github/aws3ma/jewelry-classification/blob/main/jewelry_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow import keras
import tensorflow as tf
import numpy as np
import pathlib
import cv2
import os
classification_folder='/content/drive/MyDrive/classification/'
image_size = (180, 180)


In [3]:
# cleaning images
num_skipped = 0
for folder_name in ("bagues", "boucles","bracelet","collier"):
    folder_path = os.path.join(classification_folder+'/train', folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

Deleted 0 images


In [4]:
data_dir = pathlib.Path(classification_folder+'train/')
jewelry_images_dict={
    'bagues':list(data_dir.glob('bagues/*')),
    'boucle':list(data_dir.glob('boucles/*')),
    'bracelet':list(data_dir.glob('bracelet/*')),
    'collier':list(data_dir.glob('collier/*'))
}
jewelry_labels_dict={
    'bagues':0,
    'boucle':1,
    'bracelet':2,
    'collier':3
}
x,y=[],[]
for jewelry_name, images in jewelry_images_dict.items():
  for image in images:
    img = cv2.imread(str(image))
    resized_img = cv2.resize(img,(180,180))
    x.append(resized_img)
    y.append(jewelry_labels_dict[jewelry_name])

In [5]:
from google.colab.patches import cv2_imshow
x = np.array(x)
y = np.array(y)
x_train, x_test, y_train, y_test=train_test_split(x,y,random_state=0)
x_train_scaled = x_train/255
x_test_scaled = x_test/255

In [6]:
data_aug= keras.Sequential([
                            layers.experimental.preprocessing.RandomFlip("horizontal",input_shape=(180,180,3)),
                            layers.experimental.preprocessing.RandomRotation(0.1),
                            layers.experimental.preprocessing.RandomZoom(0.1),
                            layers.experimental.preprocessing.RandomContrast([0.9,1.1])
                            
])

In [7]:
model=keras.Sequential([
                        data_aug,
                        layers.Conv2D(16, 3, padding="same", activation='relu'),
                        layers.MaxPooling2D(),
                        layers.Conv2D(32,3,padding="same",activation='relu'),
                        layers.MaxPooling2D(),
                        layers.Conv2D(64,3,padding="same",activation='relu'),
                        layers.MaxPooling2D(),
                        layers.Dropout(0.2),
                        layers.Flatten(),
                        layers.Dense(128, activation="relu"),
                        layers.Dense(4, activation='softmax'),
])
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.fit(x_train_scaled, y_train,epochs=100)
model.save(classification_folder+"checkpoints/final/")

Epoch 1/100


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


21/21 [==============================] - 24s 1s/step - loss: 1.5531 - accuracy: 0.3015
Epoch 2/100
21/21 [==============================] - 32s 2s/step - loss: 1.3161 - accuracy: 0.3682
Epoch 3/100
21/21 [==============================] - 22s 1s/step - loss: 1.1564 - accuracy: 0.5061
Epoch 4/100
21/21 [==============================] - 24s 1s/step - loss: 1.1727 - accuracy: 0.4985
Epoch 5/100
21/21 [==============================] - 22s 1s/step - loss: 1.0306 - accuracy: 0.5985
Epoch 6/100
21/21 [==============================] - 22s 1s/step - loss: 0.9501 - accuracy: 0.6439
Epoch 7/100
21/21 [==============================] - 22s 1s/step - loss: 0.9480 - accuracy: 0.5924
Epoch 8/100
21/21 [==============================] - 22s 1s/step - loss: 1.1685 - accuracy: 0.5788
Epoch 9/100
21/21 [==============================] - 22s 1s/step - loss: 1.0473 - accuracy: 0.6076
Epoch 10/100
21/21 [==============================] - 23s 1s/step - loss: 0.9723 - accuracy: 0.6136
Epoch 11/100
21/21 [=

In [8]:
model.evaluate(x_test_scaled,y_test)
model.save(classification_folder+"checkpoints/9/")

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


7/7 [==============================] - 2s 267ms/step - loss: 0.4217 - accuracy: 0.9140
INFO:tensorflow:Assets written to: /content/drive/MyDrive/classification/checkpoints/9/assets


In [ ]:
nb_bague=0
nb_boucles=0
nb_bracelet=0
nb_collier=0
test_path=classification_folder+'test/NECKLACE/'
for i in os.listdir(test_path):
  img = keras.preprocessing.image.load_img(
      test_path+i, 
      target_size=image_size,
  )
  plt.imshow(img)
  plt.show()
  img_array = keras.preprocessing.image.img_to_array(img)
  img_array = tf.expand_dims(img_array, 0)  # Create batch axis

  predictions = model.predict(img_array)
  l=predictions.tolist()[0]
  print(l)
  if l.index(max(l)) == 0:
    nb_bague+=1
    print("bague")
  if l.index(max(l)) == 1:
    nb_boucles+=1
    print("boucle")
  if l.index(max(l)) == 2:
    nb_bracelet+=1
    print("bracelet")
  if l.index(max(l)) == 3:
    nb_collier+=1
    print("collier")
  # print(f'{100*l[0]}% bagues \n{100*l[1]}% boucles \n{100*l[2]}% bracelet \n{100*l[3]}% collier')
print(f"Bagues : {nb_bague}\nBoucles : {nb_boucles}\nBracelet : {nb_bracelet}\nCollier : {nb_collier}")